In [ ]:
'''
version 2
去除 face mask
2017年10月17日15:53:14
    在预训练模型的基础下，对现实情形进行标定校正
'''

In [ ]:
import tensorflow as tf
import os
import numpy as np
from mit_eye_v2 import mit_itraker
import time
sess=tf.InteractiveSession()

In [ ]:
with tf.name_scope('PlaceHolder'):
    LeftEye=tf.placeholder(tf.float32,[None,64,64,3],name='LeftEye')
    RightEye=tf.placeholder(tf.float32,[None,64,64,3],'RightEye')
    FaceOri=tf.placeholder(tf.float32,[None,64,64,3],'FaceOri')
    #FaceMask=tf.placeholder(tf.float32,[None,25,25],'FaceMask')
    Y=tf.placeholder(tf.float32,[None,2],name='Y')
    LR=tf.placeholder(tf.float32,name='Learning_rate')
    REG_RATE=tf.placeholder(tf.float32,name='Reg_rate')
    DROPOUT=tf.placeholder(tf.float32,name='DropOut_rate')
    BN_FLAG=tf.placeholder(tf.bool,name='BatchNormalizeFlag')

In [ ]:
with tf.name_scope('MIT_Model'):
    model=mit_itraker(r'D:\Data warehouse\Face_Picture\eye_tracker_train_and_val',left_eye=LeftEye,right_eye=RightEye,face_ori=FaceOri,\
                      dropout=DROPOUT,bn_train=BN_FLAG,reg_rate=REG_RATE)
    y_pre=model.coordinate

In [ ]:
with tf.name_scope('LOSS'):
    Loss=(tf.losses.mean_squared_error(Y,y_pre))+tf.add_n(tf.get_collection('loss_w'))
    tf.summary.scalar('loss',Loss)
    ERR=tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.squared_difference(y_pre, Y), axis=1)))
    tf.summary.scalar('err',ERR)
with tf.name_scope('Trainer'):
    Trainer=tf.train.AdamOptimizer(LR).minimize(Loss)

with tf.name_scope('Logger'):
    init=tf.global_variables_initializer()

    writer_tr=tf.summary.FileWriter('./mylog/train_nofg_cal',sess.graph)
    writer_te=tf.summary.FileWriter('./mylog/val_nofg_cal')
    merge=tf.summary.merge_all()

In [ ]:
# data=model.data
model.data=None
bn_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
'''
['val_face', 'train_eye_left', 'train_y', 'train_eye_right', 'val_y', 'train_face_mask', 'train_face',
'val_eye_right', 'val_face_mask', 'val_eye_left']
'''

In [ ]:
#重新训练
sess.run(init)
saver=tf.train.Saver()

In [ ]:
#加载保存的模型，继续训练
saver=tf.train.Saver()
#saver=tf.train.Saver()
saver.restore(sess,'./model_save/model_mit_cal_1017/model_10172130.ckpt-19867')

In [ ]:
# val_left_eye,val_right_eye,val_face_matrix,val_y_cordinate=left_eye[te_index],right_eye[te_index],face_matrix[te_index],y_cordinate[te_index]
# tr_left_eye,tr_right_eye,tr_face_matrix,tr_y_cordinate=left_eye[tr_index],right_eye[tr_index],face_matrix[tr_index],y_cordinate[tr_index]

In [ ]:
#130 个loop用时1分钟 无BN,dropout
#5000 个loop用时1h 有BN,dropout
batch_size=128
dp_=0.5
lr_=1e-3
rg_rate_=1e-4
s_time=time.time()
best_val_err=100
err_delta=0.05
for i_ in range(5000*15):
    mask=np.random.choice(range(tr_left_eye.shape[0]),batch_size,replace=False)
    face_=tr_face_matrix[mask]
    left_eye_,right_eye_=tr_left_eye[mask],tr_right_eye[mask]
    y_=tr_y_cordinate[mask]
    feed_dict={LeftEye:left_eye_,  RightEye:right_eye_,  FaceOri:face_,  \
              Y:y_,LR:lr_,  DROPOUT:dp_,  BN_FLAG:True,  REG_RATE:rg_rate_}
    sess.run([Trainer,bn_ops],feed_dict=feed_dict)
    if i_%5==0:
        feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,\
              Y:y_,LR:lr_,DROPOUT:1.,BN_FLAG:False,REG_RATE:rg_rate_}
        loss_,m_,_=sess.run([Loss,merge,bn_ops],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i_)
        print('epoch:{},loss:{}'.format(i_,loss_))
    if i_%10==0:
        #mask=np.random.choice(range(5000),batch_size,replace=False)
        face_=val_face_matrix
        left_eye_,right_eye_=val_left_eye,val_right_eye
        y_=val_y_cordinate
        feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,\
                  Y:y_,LR:lr_,DROPOUT:1.,BN_FLAG:False,REG_RATE:rg_rate_}
        loss_,err_,m_,_=sess.run([Loss,ERR,merge,bn_ops],feed_dict=feed_dict)
        writer_te.add_summary(m_,i_)
        print('--epoch:{},loss:{},error:{}'.format(i_,loss_,err_))
        #保存模型
#         if err_<0:
#             if err_+err_delta<best_val_err:
#                 best_val_err=err_
#                 err_val=str(int(err_*100))
#                 saver.save(sess,'./model_save/model_mit/model_err'+err_val+'.ckpt',global_step=i_)
#                 print('@@@epoch:{},best err:{},model save'.format(i_,best_val_err))
#     if i_%(5000)==0:
#         lr_=max(1e-5,lr_*0.9)
#         print("**epoch:{},lr:{}".format(i_,lr_))
e_time=time.time()
run_time=e_time-s_time
print('Trainning Time:{:.0f} h,{:.0f} min,{:.2f} sec'.format(run_time//3600,(run_time%3600)//60,run_time%60))


In [ ]:
saver.save(sess,'./model_save/model_mit_cal_1017/model_10172130'+'.ckpt',global_step=i_)

In [ ]:
import cv2
import time

In [ ]:
#%debug
cap=cv2.VideoCapture(0)
s_time=time.time()
while True:
    ret,fram=cap.read()
    if ret:
        face_op=drew_face_eye(fram)
        if face_op == None:
            #print('fail')
            continue
        img_,f,e=face_op
        left_eye_=cv2.resize(e[0],(64,64)).reshape((1,64,64,3))
        right_eye_=cv2.resize(e[1],(64,64)).reshape((1,64,64,3))
        face_=cv2.resize(f,(64,64)).reshape((1,64,64,3))
        
        feed_dict={LeftEye:left_eye_,RightEye:right_eye_,FaceOri:face_,\
                 DROPOUT:1.,BN_FLAG:False}
        y_cori=sess.run(y_pre,feed_dict=feed_dict)
        print(y_cori)
        cv2.imshow('asd',fram)
        if cv2.waitKey()&0xff==27:
            break
    if (time.time()-s_time)>10:
        print('time out')
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
xy_cor=

In [ ]:
/****************************************************************************/
获取标定图片
*****************************************************************************/

In [ ]:
import cv2
import interface
import os
import numpy as np

In [ ]:

def drew_face_eye(img):
        '''
        检测用户当前环境是否能够比较好的识别面部特征
        辨识出脸部以及眼部，标记出来
        :param img:
        :return:
        '''
        face_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(r'D:\Proj_DL\Code\Proj_EyeTraker\haarcascade_eye.xml')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3,5)
        #确保画面中只有一个人脸识别出来
        if(len(faces)!=1):
            return None
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x-10,y-10),(x+w+10,y+h+10),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            #保存脸部图像
            face_mat=roi_color.copy()
            #检测视频中脸部的眼睛，并用vector保存眼睛的坐标、大小（用矩形表示）
            eyes = eye_cascade.detectMultiScale(roi_gray,scaleFactor=1.3, minNeighbors=7, minSize=(50, 50),
                                                flags=cv2.CASCADE_SCALE_IMAGE)
            if len(eyes)!= 2:
                return None
            eye_mat=[]
            for e in eyes:
                xe,ye,we,he=e
                eye_mat.append(face_mat[ye:ye+he,xe:xe+we])
                cv2.rectangle(roi_color,(xe-3,ye-3),(xe+we+3,ye+he+3),(0,0,255),2)
        return  (img,face_mat,eye_mat)

In [ ]:
cal_obj=interface.Cal_Interface(line_num=4,cal_time=16*2)

In [ ]:
cal_obj.starcalibrate()

In [ ]:
file_root=r'D:\Proj_DL\Code\Proj_EyeTraker\Proj_iTraker\MIT_iTraker\calimg_file'
img_names=os.listdir(file_root)

In [ ]:
#array([ 1.70154929, -1.93875003], dtype=float32) 标签格式
y_cordinate=[]
left_eye=[]
right_eye=[]
face_matrix=[]
for f in img_names:
    x_,y_=f[:-4].split('_')[-2:]
    x_,y_=int(x_[-1]),int(y_[-1])
    img=cv2.imread(os.path.join(file_root,f))
    face_op=drew_face_eye(img)
    if face_op == None:
        #print('fail')
        continue
    img_,f,e=face_op
    y_cordinate.append([x_,y_])
    
    left_eye.append(cv2.resize(e[0],(64,64)))
    right_eye.append(cv2.resize(e[1],(64,64)))
    face_matrix.append(cv2.resize(f,(64,64)))
    #cv2.imshow('asd',img_)
    if cv2.waitKey(1)&0xff == 27:
        break
cv2.destroyAllWindows()
y_cordinate=np.array(y_cordinate).astype('float32')

left_eye=np.array(left_eye)
left_eye=left_eye.astype('float32')-np.mean(left_eye,0)
left_eye/=255.0

right_eye=np.array(right_eye)
right_eye=right_eye.astype('float32')-np.mean(right_eye,0)
right_eye/=255.0

face_matrix=np.array(face_matrix)
face_matrix=face_matrix.astype('float32')-np.mean(face_matrix,0)
face_matrix/=255.0



In [ ]:
index=np.arange(face_matrix.shape[0])
np.random.shuffle(index)
tr_index=index[:int(index.shape[0]*0.7)]
te_index=index[int(index.shape[0]*0.7):]
val_left_eye,val_right_eye,val_face_matrix,val_y_cordinate=left_eye[te_index],right_eye[te_index],face_matrix[te_index],y_cordinate[te_index]
tr_left_eye,tr_right_eye,tr_face_matrix,tr_y_cordinate=left_eye[tr_index],right_eye[tr_index],face_matrix[tr_index],y_cordinate[tr_index]
y_cordinate=[]
left_eye=[]
right_eye=[]
face_matrix=[]

In [ ]:
tr_left_eye.shape,val_left_eye.shape